In [115]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
from sklearn.metrics import precision_score, recall_score, f1_score


file_path = 'raw.csv'
df = pd.read_csv(file_path)

df.head()


,title,price,location,link_item,price_before_discount,discount_percent,sold_quantity,rating,details
0,adidas Quần vợt Giày Park Street Nam trắng IG9848,1.350.000 ₫,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,1.800.000 ₫,"LazFlash Sale,kết thúc sau 13h",40 Đã bán,3.6,Dáng regular fit\nCó dây giày\nThân giày bằng ...
1,adidas Quần vợt Giày Tennis Barricade 13 Nam t...,4.000.000 ₫,Bình Dương,https://www.lazada.vn/products/adidas-quan-vot...,NaN,NaN,5 Đã bán,4.5,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
2,adidas Quần vợt Giày Park Street Nam trắng IG9848,1.350.000 ₫,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,1.800.000 ₫,"LazFlash Sale,kết thúc sau 13h",14 Đã bán,3.9,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
3,adidas Quần vợt Giày Advantage 20 Nam trắng IG...,1.500.000 ₫,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,2.000.000 ₫,"LazFlash Sale,kết thúc sau 13h",42 Đã bán,3.8,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
4,adidas Chạy Giày Galaxy 6 Nam Đen GW4138,1.260.000 ₫,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,1.800.000 ₫,Voucher giảm 30%,44 Đã bán,3.4,Dáng regular fit\r\nCó dây giày\r\nThân giày b...


PREPROCESS DATA

In [116]:
# Check NULL
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  915 non-null    object 
 1   price                  915 non-null    object 
 2   location               915 non-null    object 
 3   link_item              915 non-null    object 
 4   price_before_discount  806 non-null    object 
 5   discount_percent       806 non-null    object 
 6   sold_quantity          848 non-null    object 
 7   rating                 915 non-null    float64
 8   details                913 non-null    object 
dtypes: float64(1), object(8)
memory usage: 64.5+ KB


title                      0
price                      0
location                   0
link_item                  0
price_before_discount    109
discount_percent         109
sold_quantity             67
rating                     0
details                    2
dtype: int64

In [ ]:
# Preprocess numeric data
## Transform into string
df['price'] = df['price'].astype(str)   
df['price_before_discount'] = df['price_before_discount'].astype(str)
df['discount_percent'] = df['discount_percent'].astype(str)

## Remove not numeric values
df['price'] = df['price'].str.replace(r'[^\d]', '', regex=True)
df['price_before_discount'] = df['price_before_discount'].str.replace(r'[^\d]', '', regex=True)
df['discount_percent'] = df['discount_percent'].str.extract(r'(\d+%)')
df['sold_quantity'] = df['sold_quantity'].str.extract(r'(\d+)') 

## Change back to numeric value
df['sold_quantity'] = pd.to_numeric(df['sold_quantity'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['price_before_discount'] = pd.to_numeric(df['price_before_discount'], errors='coerce')

# Convert 'discount_percent' to numeric and remove "%"
df['discount_percent'] = df['discount_percent'].str.replace('%', '').astype(float)

# Handle missing values
df['price_before_discount'].fillna(df['price'], inplace=True)
df['sold_quantity'].fillna(0, inplace=True)
df.loc[df['discount_percent'].isna(), 'discount_percent'] = (
    ((df['price_before_discount'] - df['price']) / df['price_before_discount']) * 100
)

df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  915 non-null    object 
 1   price                  915 non-null    int64  
 2   location               915 non-null    object 
 3   link_item              915 non-null    object 
 4   price_before_discount  915 non-null    float64
 5   discount_percent       915 non-null    float64
 6   sold_quantity          915 non-null    float64
 7   rating                 915 non-null    float64
 8   details                913 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 64.5+ KB


(None,
                                                title    price    location  \
 0  adidas Quần vợt Giày Park Street Nam trắng IG9848  1350000  Bình Dương   
 1  adidas Quần vợt Giày Tennis Barricade 13 Nam t...  4000000  Bình Dương   
 2  adidas Quần vợt Giày Park Street Nam trắng IG9848  1350000  Bình Dương   
 3  adidas Quần vợt Giày Advantage 20 Nam trắng IG...  1500000  Bình Dương   
 4           adidas Chạy Giày Galaxy 6 Nam Đen GW4138  1260000  Bình Dương   
 
                                            link_item  price_before_discount  \
 0  https://www.lazada.vn/products/chi-11-1211-mua...              1800000.0   
 1  https://www.lazada.vn/products/adidas-quan-vot...              4000000.0   
 2  https://www.lazada.vn/products/chi-11-1211-mua...              1800000.0   
 3  https://www.lazada.vn/products/chi-11-1211-mua...              2000000.0   
 4  https://www.lazada.vn/products/chi-11-1211-mua...              1800000.0   
 
    discount_percent  sold_quantity  rati

In [ ]:
# Giảm chiều dữ liệu
## Tính ma trận tương quan
# Lọc ra các cột chỉ chứa giá trị số
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Tính toán ma trận tương quan
corr_matrix = numeric_df.corr()

print("Ma trận tương quan:")
print(corr_matrix)

Ma trận tương quan:
                          price  price_before_discount  discount_percent  \
price                  1.000000               0.966631          0.104520   
price_before_discount  0.966631               1.000000          0.308648   
discount_percent       0.104520               0.308648          1.000000   
sold_quantity         -0.077602              -0.123256         -0.141298   
rating                -0.039625              -0.103242         -0.285610   

                       sold_quantity    rating  
price                      -0.077602 -0.039625  
price_before_discount      -0.123256 -0.103242  
discount_percent           -0.141298 -0.285610  
sold_quantity               1.000000  0.309422  
rating                      0.309422  1.000000  


In [119]:
df = df.drop(columns=['price_before_discount'])
df.head()

,title,price,location,link_item,discount_percent,sold_quantity,rating,details
0,adidas Quần vợt Giày Park Street Nam trắng IG9848,1350000,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,40.0,3.6,Dáng regular fit\nCó dây giày\nThân giày bằng ...
1,adidas Quần vợt Giày Tennis Barricade 13 Nam t...,4000000,Bình Dương,https://www.lazada.vn/products/adidas-quan-vot...,0.0,5.0,4.5,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
2,adidas Quần vợt Giày Park Street Nam trắng IG9848,1350000,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,14.0,3.9,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
3,adidas Quần vợt Giày Advantage 20 Nam trắng IG...,1500000,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,42.0,3.8,Dáng regular fit\r\nCó dây giày\r\nThân giày b...
4,adidas Chạy Giày Galaxy 6 Nam Đen GW4138,1260000,Bình Dương,https://www.lazada.vn/products/chi-11-1211-mua...,30.0,44.0,3.4,Dáng regular fit\r\nCó dây giày\r\nThân giày b...


In [120]:
# Preprocess text data
## Remove NULL data
df.dropna(), 
df = df.drop(columns=['location'])
df.isnull().sum()
df.head()

# Normalize text data
## Lowercase
df['title'] = df['title'].str.lower()
df['details'] = df['details'].str.lower()

## Remove Puntuation
df['title'] = df['title'].str.replace(r'[^\w\s]', '', regex=True)
df['details'] = df['details'].str.replace(r'[^\w\s]', '', regex=True)

## Remove extra whitespaces
df['title'] = df['title'].str.strip()  # Loại bỏ khoảng trắng đầu và cuối
df['title'] = df['title'].str.replace(r'\s+', ' ', regex=True)  # Thay nhiều khoảng trắng bằng một khoảng trắng
df['details'] = df['details'].str.strip()  # Loại bỏ khoảng trắng đầu và cuối
df['details'] = df['details'].str.replace(r'\s+', ' ', regex=True)  # Thay nhiều khoảng trắng bằng một khoảng trắng

df.head()


,title,price,link_item,discount_percent,sold_quantity,rating,details
0,adidas quần vợt giày park street nam trắng ig9848,1350000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,40.0,3.6,dáng regular fit có dây giày thân giày bằng da...
1,adidas quần vợt giày tennis barricade 13 nam t...,4000000,https://www.lazada.vn/products/adidas-quan-vot...,0.0,5.0,4.5,dáng regular fit có dây giày thân giày bằng vả...
2,adidas quần vợt giày park street nam trắng ig9848,1350000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,14.0,3.9,dáng regular fit có dây giày thân giày bằng da...
3,adidas quần vợt giày advantage 20 nam trắng ig...,1500000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,42.0,3.8,dáng regular fit có dây giày thân giày bằng da...
4,adidas chạy giày galaxy 6 nam đen gw4138,1260000,https://www.lazada.vn/products/chi-11-1211-mua...,30.0,44.0,3.4,dáng regular fit có dây giày thân giày bằng vả...


PROCESS DATA - BUILD MODEL

In [121]:
# Generate column 'id'
df.insert(0, 'id', range(1, len(df) + 1))
df.head()

,id,title,price,link_item,discount_percent,sold_quantity,rating,details
0,1,adidas quần vợt giày park street nam trắng ig9848,1350000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,40.0,3.6,dáng regular fit có dây giày thân giày bằng da...
1,2,adidas quần vợt giày tennis barricade 13 nam t...,4000000,https://www.lazada.vn/products/adidas-quan-vot...,0.0,5.0,4.5,dáng regular fit có dây giày thân giày bằng vả...
2,3,adidas quần vợt giày park street nam trắng ig9848,1350000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,14.0,3.9,dáng regular fit có dây giày thân giày bằng da...
3,4,adidas quần vợt giày advantage 20 nam trắng ig...,1500000,https://www.lazada.vn/products/chi-11-1211-mua...,25.0,42.0,3.8,dáng regular fit có dây giày thân giày bằng da...
4,5,adidas chạy giày galaxy 6 nam đen gw4138,1260000,https://www.lazada.vn/products/chi-11-1211-mua...,30.0,44.0,3.4,dáng regular fit có dây giày thân giày bằng vả...


In [ ]:
# Make ground truth
ground_truth = pd.DataFrame({
    'id': [1, 125, 497],  # ID sản phẩm đầu vào
    'relevant_items': [[2, 76, 791], [799, 98, 807], [476, 362, 407]]  # Sản phẩm phù hợp
})

# Use TF-IDF to vectorize text data
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_title = tfidf_vectorizer.fit_transform(df['title'].fillna(''))
tfidf_details = tfidf_vectorizer.fit_transform(df['details'].fillna(''))

# Use MinMaxScaler to norrmalize numeric data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['price', 'discount_percent', 'rating', 'sold_quantity']].fillna(0))

# Combine text and number feature
combined_features = hstack([tfidf_title, tfidf_details, scaled_features])
combined_features = combined_features.tocsr()  # Chuyển đổi sang csr_matrix

# Bulid system model with KNN
knn = NearestNeighbors(n_neighbors=6, metric='manhattan')  # Sử dụng manhattan similarity trong KNN
knn.fit(combined_features)

def recommend(product_id, num_recommendations=5):
    # Find target product
    product_index = df[df['id'] == product_id].index[0]

    # K nearest
    distances, indices = knn.kneighbors(combined_features[product_index], n_neighbors=num_recommendations + 1)

    # Output
    recommendations = df.iloc[indices.flatten()[1:]]

    return recommendations[['title', 'link_item', 'price', 'rating', 'sold_quantity']]

In [ ]:
def wrap_text(text, max_length=50):
    """
    Chia chuỗi dài thành các đoạn nhỏ hơn với độ dài tối đa là max_length,
    thêm ký tự xuống dòng giữa các đoạn.
    """
    return '\n'.join([text[i:i + max_length] for i in range(0, len(text), max_length)])

def display_recommendations(product_id, num_recommendations=5):
    try:
        input_product = df[df['id'] == product_id]
        if input_product.empty:
            print(f"Sản phẩm với ID {product_id} không tồn tại trong dataset.")
            return

        # Display input
        print("Thông tin sản phẩm đã chọn:")
        input_product_display = input_product[['title','sold_quantity', 'price', 'rating', 'link_item']].copy()
        input_product_display.loc[:, 'title'] = input_product_display['title'].apply(lambda x: wrap_text(str(x), max_length=50))
        input_product_display.loc[:, 'link_item'] = input_product_display['link_item'].apply(lambda x: wrap_text(str(x), max_length=50))
        print(tabulate(input_product_display, headers='keys', tablefmt='fancy_grid'))

        # Take output infor
        recommendations = recommend(product_id, num_recommendations)
        recommendations_display = recommendations[['title','sold_quantity', 'price', 'rating', 'link_item']].copy()
        recommendations_display.loc[:, 'title'] = recommendations_display['title'].apply(lambda x: wrap_text(str(x), max_length=50))
        recommendations_display.loc[:, 'link_item'] = recommendations_display['link_item'].apply(lambda x: wrap_text(str(x), max_length=50))

        # Display output
        print("\nTop sản phẩm được gợi ý:")
        print(tabulate(recommendations_display, headers='keys', tablefmt='fancy_grid'))

    except Exception as e:
        print(f"Có lỗi xảy ra: {e}")

# Input
try:
    product_id = int(input("Nhập ID sản phẩm (1 - 915): "))
    display_recommendations(product_id)
except Exception as e:
    print("Có lỗi xảy ra:", e)


Thông tin sản phẩm đã chọn:
╒═════╤═══════════════════════════════════════════════════╤═════════════════╤═════════╤══════════╤════════════════════════════════════════════════════╕
│     │ title                                             │   sold_quantity │   price │   rating │ link_item                                          │
╞═════╪═══════════════════════════════════════════════════╪═════════════════╪═════════╪══════════╪════════════════════════════════════════════════════╡
│ 100 │ adidas chạy giày response super 20 nam đen g58068 │             154 │ 1875000 │      3.7 │ https://www.lazada.vn/products/chi-11-1211-mua-3-g │
│     │                                                   │                 │         │          │ iam-50-adidas-chay-giay-response-super-20-nam-den- │
│     │                                                   │                 │         │          │ g58068-i1479103118.html                            │
╘═════╧═══════════════════════════════════════════════════╧═